In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np

In [11]:
#Reads the data into a dataframe and renames the columns to remove unnecessary numbers 
data_df = pd.read_excel("RNASeqData.xlsx")
data_df = data_df.rename(columns=lambda x: re.sub("w_[0-9]+_O", "w_O",x))
data_df = data_df.rename(columns=lambda x: re.sub("m_[0-9]+_O", "m_O",x))

In [ ]:
columns = list(data_df.columns) 

In [ ]:
norm_cols = [x for x in columns if "Norm" in x]
norm_cols = [x for x in norm_cols if "HC" not in x]
norm_cols_base = [x for x in norm_cols if "Base" in x]
norm_cols_v2 = [x for x in norm_cols if "V2" in x]

df_norm_base = data_df[norm_cols_base]
df_norm_base = df_norm_base.rename(columns=lambda x: re.sub("Norm_Orkambi_0", "",x))
df_norm_base = df_norm_base.rename(columns=lambda x: re.sub("_Base", "",x))
df_norm_base = df_norm_base.sort_index(axis=1)
df_norm_base = df_norm_base.drop(columns=['02'])

df_norm_v2 = data_df[norm_cols_v2]
df_norm_v2 = df_norm_v2.rename(columns=lambda x: re.sub("Norm_Orkambi_0", "",x))
df_norm_v2 = df_norm_v2.rename(columns=lambda x: re.sub("_V2", "",x))
df_norm_v2 = df_norm_v2.sort_index(axis=1)
df_norm_v2 = df_norm_v2.drop(columns=['19'])

In [ ]:
from statistics import mean
def get_mean_diff(base, v2):
  diff = [base[i] - v2[i] for i in range(len(base))] 
  mean_diff = mean(diff)
  return mean_diff

In [ ]:
from random import randrange
def get_avg_diff_count(observed_avg_diff, base, v2, num_shuffles):
  base_copy = base.copy()
  v2_copy = v2.copy()
  count = 0
  avg_diffs = []
  for i in range(num_shuffles):
    # if (i>0 and i%1000 == 0):
    #   print("Done: ", i)
    idx = randrange(len(base))-1
    temp = base_copy[idx]
    base_copy[idx] = v2_copy[idx]
    v2_copy[idx] = temp
    avg_diff = get_mean_diff(base_copy,v2_copy)
    avg_diffs.append(avg_diff)
    if observed_avg_diff < 0 and avg_diff <= observed_avg_diff:
      count += 1
    elif observed_avg_diff >= 0 and avg_diff >= observed_avg_diff:
      count += 1
  return count, np.asarray(avg_diffs)


In [ ]:
#draw histogram

def draw_hist(d, observed):
    print(observed)
    hist,bin_edges = np.histogram(d,bins =70)    
    plt.figure(figsize=[8,8])
    plt.bar(bin_edges[:-1], hist, width = 0.1, color='#0504aa',alpha=0.7)
    plt.xlim(min(min(bin_edges),-4), max(max(bin_edges),4))
    plt.grid(axis='y', alpha=0.75)
    plt.xlabel('Value',fontsize=15)
    plt.ylabel('Frequency',fontsize=15)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.ylabel('Frequency',fontsize=15)
    plt.title('Normal Distribution Histogram',fontsize=15)
    plt.axvline(x=observed, color='r', linestyle='dashed', linewidth=2)
    plt.show()

In [ ]:
def avg_diff_sig_test(df_norm_base, df_norm_v2):
    num_shuffles = 10000
    num_genes = df_norm_base.shape[0]
    all_p_sig_values = []
    for i in range(num_genes):
      print(i)
      base = list(df_norm_base.loc[i, :])
      v2 = list(df_norm_v2.loc[i, :])
      observed_avg_diff = get_mean_diff(base, v2)
      count, avg_diffs = get_avg_diff_count(observed_avg_diff, base, v2, num_shuffles)
      # draw_hist(avg_diffs, observed_avg_diff)
        
      ######################################
      #
      # Output
      #
      ######################################
      # print("**********Gene: ", i, "**********")

      # print ("Observed avg of differences: %.2f" % observed_avg_diff)
      # print (count, "out of", num_shuffles, "experiments had a difference of two means", end=" ")
      # if observed_avg_diff < 0:
      #     print ("less than or equal to", end=" ")
      # else:
      #     print ("greater than or equal to", end=" ")
      # print ("%.2f" % observed_avg_diff, ".")
      # print ("The chance of getting a difference of two means", end=" ")
      # if observed_avg_diff < 0:
      #     print ("less than or equal to", end=" ")
      # else:
      #     print ("greater than or equal to", end=" ")
      # print ("%.2f" % observed_avg_diff, "is", (count / float(num_shuffles)), "\n")
      all_p_sig_values.append(count / float(num_shuffles))
    return all_p_sig_values

In [ ]:
norm_p_values = avg_diff_sig_test(df_norm_base, df_norm_v2)

In [ ]:
import pickle
pickle.dump( norm_p_values, open( "norm_p_values.p", "wb" ) )

**Bonferroni correction**

In this method, we divide the threshold by the number of genes. So, if we take a default threshold of 5% (or 0.05), now our threshold would be `0.05/len(norm_p_values)`

In [9]:
import pickle
norm_p_values = pickle.load( open( "norm_p_values.p", "rb" ) )
count_significant_genes_bonferroni = 0
for i, ele in enumerate(norm_p_values):
  if ele < (0.05/len(norm_p_values)):
    count_significant_genes_bonferroni += 1
    print(i+1)
print("Number of significant genes: ", count_significant_genes_bonferroni)

188
2066
3290
5750
7204
8384
10516
13503
Number of significant genes:  8


**Benjamini–Hochberg procedure**

 Put the individual P values in order, from smallest to largest. The smallest P value has a rank of i=1, then next smallest has i=2, etc. Compare each individual P value to its Benjamini-Hochberg critical value, (i/m)Q, where i is the rank, m is the total number of tests, and Q is the false discovery rate you choose. The largest P value that has P<(i/m)Q is significant, and all of the P values smaller than it are also significant, even the ones that aren't less than their Benjamini-Hochberg critical value.

In [5]:
def benjamin_hochberg_corrected_p_values(p_values, fdr):
  num_genes = len(p_values)
  import numpy as np
  sorted_indices = np.argsort(p_values)
  sorted_p_values = sorted(p_values)
  benjamini_hochberg_critical_values = [((i+1)/num_genes)*fdr for i in range(num_genes)]
  for i, p_value in reversed(list(enumerate(sorted_p_values))):
    if p_value < benjamini_hochberg_critical_values[i]:
      break
  return i, sorted_indices

In [6]:
import pickle
norm_p_values = pickle.load( open( "norm_p_values.p", "rb" ) )
i, sorted_indices = benjamin_hochberg_corrected_p_values(norm_p_values, 0.1)
print("Number of significant genes: ", i+1)

Number of significant genes:  8


In [7]:
significant_genes = []
for i in range(8):
  significant_genes.append((sorted_indices[i]+1))


significant_genes

[8384, 2066, 13503, 7204, 188, 10516, 5750, 3290]

In [12]:
print(data_df.loc[data_df['RowID'].isin(significant_genes)][['RowID', 'ID', 'Description']])

       RowID            ID                                        Description
187      188        ADAM22                    ADAM metallopeptidase domain 22
2065    2066         CENPM                               centromere protein M
3289    3290       EEF1DP3  eukaryotic translation elongation factor 1 del...
5749    5750         KITLG                                         KIT ligand
7203    7204  LOC105378698                                                NaN
8383    8384       MIR3939                                      microRNA 3939
10515  10516         PSMD1               proteasome 26S subunit, non-ATPase 1
13502  13503       TMEM181                          transmembrane protein 181
